In [1]:
import tensorflow as tf
import os

model_dirs = [
    os.path.join(os.curdir, "model", name)
    for name in sorted(os.listdir(os.path.join(os.curdir, "model")))
]
model_dirs


2022-11-17 15:10:47.151244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 15:10:47.558078: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 15:10:48.608640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-17 15:10:48.608735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

['./model/BASEL',
 './model/BUDAPEST',
 './model/DE',
 './model/DRESDEN',
 './model/DUSSELDORF',
 './model/HEATHROW',
 './model/KASSEL',
 './model/LJUBLJANA',
 './model/MAASTRICHT',
 './model/MALMO',
 './model/MONTELIMAR',
 './model/MUENCHEN',
 './model/OSLO',
 './model/PERPIGNAN',
 './model/ROMA',
 './model/SONNBLICK',
 './model/STOCKHOLM',
 './model/TOURS']

In [2]:
models = [tf.keras.models.load_model(model_dir) for model_dir in model_dirs]
models[1].summary()

2022-11-17 15:10:50.144791: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-17 15:10:50.185000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-17 15:10:50.185583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-17 15:10:50.186964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Model: "BUDAPEST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 6, 7)]            0         
                                                                 
 lstm_2 (LSTM)               (None, 6, 16)             1536      
                                                                 
 lstm_3 (LSTM)               (None, 16)                2112      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,665
Trainable params: 3,665
Non-trainable params: 0
_________________________________________________________________


In [3]:
import pandas as pd

CSV_DIR = os.path.join(os.curdir, "dataset", "sorted")
csv_filenames = sorted(os.listdir(CSV_DIR))
csv_dataset = [
    pd.read_csv(os.path.join(CSV_DIR, filename)) for filename in csv_filenames
]
print(len(csv_dataset))

18


In [4]:

TRAIN_END_INDEX = 2922
VALIDATION_END_INDEX = 3288



def get_dataset(
    dataset: pd.DataFrame,
    sequence_length,
    batch_size,
    train_end_index,
    validation_end_index,
):
    data = dataset
    for col in dataset.columns:
        if "temp_mean" in col:
            target = pd.DataFrame(dataset[col])
            data = data.drop(columns=["DATE"])
            break
    scaled_data = (data - data.mean(axis=0)) / data.std(axis=0)
    train_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=0,
        end_index=train_end_index,
    )
    validation_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=train_end_index,
        end_index=validation_end_index,
    )
    test_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=validation_end_index,
    )
    return (train_dataset, validation_dataset, test_dataset)


sequence_length = 6

batch_dataset = [
    get_dataset(csv_data, sequence_length, 32, TRAIN_END_INDEX, VALIDATION_END_INDEX)
    for csv_data in csv_dataset
]

print(len(batch_dataset))
print(batch_dataset[1])
for data, label in batch_dataset[0][0]:
    print(data[0])
    print(label[0])
    break


18
(<BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>, <BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>, <BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>)
tf.Tensor(
[[ 1.34424888  1.34682202 -1.20851298 -0.38199123 -1.07646018 -1.09549112
  -0.80998751 -1.33429094]
 [ 1.15869866  1.7487146  -1.15505695 -0.43793346 -1.07646018 -1.00108475
  -0.64465741 -1.23109558]
 [ 0.602048    1.69847803 -0.8877768  -0.43793346 -0.22197883 -1.18989749
  -1.03543765 -1.23109558]
 [ 0.41649778  1.04540259 -0.74879112  0.2147259   0.51703206 -0.96062487
  -0.97531762 -0.92150951]
 [ 1.43702399  0.8444563  -0.87708559 -0.30740159 -0.22197883 -0.67740576
  -0.47932731 -0.79538185]
 [ 0.97314844  0.8193380

In [5]:
model_losses = dict()
for model, batch_data in zip(models, batch_dataset):
    loss, mae = model.evaluate(batch_data[2])
    model_losses[model.name] = {"loss": loss, "MAE": mae}
model_losses

2022-11-17 15:11:49.009854: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


12/12 [==============================] - 1s 10ms/step - loss: 4.4037 - MAE: 1.6623


{'BASEL_model': {'loss': 3.9772305488586426, 'MAE': 1.562368631362915},
 'BUDAPEST_model': {'loss': 3.9238288402557373, 'MAE': 1.5208358764648438},
 'DE_model': {'loss': 3.1648616790771484, 'MAE': 1.4026259183883667},
 'DRESDEN_model': {'loss': 4.364295959472656, 'MAE': 1.6324210166931152},
 'DUSSELDORF_model': {'loss': 3.765521764755249, 'MAE': 1.53096342086792},
 'HEATHROW_model': {'loss': 1.6749207973480225, 'MAE': 1.0141819715499878},
 'KASSEL_model': {'loss': 3.737462282180786, 'MAE': 1.4786101579666138},
 'LJUBLJANA_model': {'loss': 4.896814346313477, 'MAE': 1.6801944971084595},
 'MAASTRICHT_model': {'loss': 4.179484844207764, 'MAE': 1.5893174409866333},
 'MALMO_model': {'loss': 2.7623682022094727, 'MAE': 1.274404525756836},
 'MONTELIMAR_model': {'loss': 3.695478916168213, 'MAE': 1.5372430086135864},
 'MUENCHEN_model': {'loss': 4.781796455383301, 'MAE': 1.7042547464370728},
 'OSLO_model': {'loss': 3.1108672618865967, 'MAE': 1.337936282157898},
 'PERPIGNAN_model': {'loss': 4.43485

In [6]:
model_loss_df = pd.DataFrame(model_losses)
model_loss_df

,BASEL_model,BUDAPEST_model,DE_model,DRESDEN_model,DUSSELDORF_model,HEATHROW_model,KASSEL_model,LJUBLJANA_model,MAASTRICHT_model,MALMO_model,MONTELIMAR_model,MUENCHEN_model,OSLO_model,PERPIGNAN_model,ROMA_model,SONNBLICK_model,STOCKHOLM_model,TOURS_model
loss,3.977231,3.923829,3.164862,4.364296,3.765522,1.674921,3.737462,4.896814,4.179485,2.762368,3.695479,4.781796,3.110867,4.434859,2.480039,4.144467,3.039230,4.403682
MAE,1.562369,1.520836,1.402626,1.632421,1.530963,1.014182,1.478610,1.680194,1.589317,1.274405,1.537243,1.704255,1.337936,1.645561,1.142359,1.581006,1.353601,1.662291


In [7]:
model_loss_df.mean(axis=1)

loss    3.696512
MAE     1.480565
dtype: float64